In [1]:
from meshplot import plot
import numpy as np
import igl
import time, math

In [2]:
from numba import cuda, jit, prange, njit, int32, float32

In [3]:
v, f = igl.read_triangle_mesh('data/cylinder.obj')

In [4]:
plot(v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [5]:
def voxel_points(axes):
    """generate n-dimensional grid coordinates from the given set of (x, y, z...) values"""
    ndims = len(axes)
    dims = [len(x) for x in axes]
    indices = np.indices(dims)
    points = np.array([axis[index] for axis, index in zip(axes, indices)])
    points = points.reshape([ndims, np.prod(dims)])
    return points.T.copy()

In [6]:
def sample_mesh_interior(V, F, gridLength, dtype=np.float32):
    minPt = np.min(V, 0)
    maxPt = np.max(V, 0)
    q = voxel_points((np.arange(minPt[0], maxPt[0] + gridLength, gridLength, dtype=dtype),
                       np.arange(minPt[1], maxPt[1] + gridLength, gridLength, dtype=dtype),
                         np.arange(minPt[2], maxPt[2] + gridLength, gridLength, dtype=dtype)))
    #print(q)
    w = igl.fast_winding_number_for_meshes(V.astype(dtype), F, q)
    #print(w.shape)
    return q[w > 0.5]

@njit (parallel=True)
def compute_inertia_tensor(points, pointmass):
    inertia_tensor = np.zeros((3, 3), dtype=points.dtype)
    N = points.shape[0]
    for i in prange(N):
        pointCM = points[i,:]
        moments = pointCM * pointCM
        inertia_tensor[0, 0] += moments[1] + moments[2]
        inertia_tensor[1, 1] += moments[0] + moments[2]
        inertia_tensor[2, 2] += moments[0] + moments[1]
        inertia_tensor[0, 1] = pointCM[0] * pointCM[1]
        inertia_tensor[0, 2] = pointCM[0] * pointCM[2]
        inertia_tensor[1, 2] = pointCM[1] * pointCM[2]
    inertia_tensor *= pointmass
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]
    return inertia_tensor

In [7]:
class PhysObject:
    def __init__(self, V, F, gridDim, density=1):
        self.points = sample_mesh_interior(V, F, gridDim)
        pointmass = density * gridDim ** 3
        cm = np.mean(self.points, 0)
        self.points -= cm
        self.mass = pointmass * self.points.shape[0]
        self.inertia_tensor = compute_inertia_tensor(self.points, pointmass)

gridDim = 0.02
start = time.time()
physobj = PhysObject(v, f, gridDim)
end = time.time()
print(end-start,'seconds with',physobj.points.shape[0],'points')

In [17]:
cell_size = 2

@cuda.jit(device=True)
def hamilton_product(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@cuda.jit
def populate_grid_kernel(points1, pose1, points2, pose2, grid, minPt, gridDim):
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(7, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(7):
            pose[j] = pose1[j]
        offset = 0
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(7):
            pose[j] = pose2[j]
        offset = cell_size
    else:
        return
    
    #rotate
    hamilton_product(pose, transpoint, transpoint)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product(transpoint, pose, transpoint)
    #translate
    transpoint[1] += pose[4] - minPt[0]
    transpoint[2] += pose[5] - minPt[1]
    transpoint[3] += pose[6] - minPt[2]
    transpoint[1] /= gridDim
    transpoint[2] /= gridDim
    transpoint[3] /= gridDim
    px = int(math.floor(transpoint[1]))
    py = int(math.floor(transpoint[2]))
    pz = int(math.floor(transpoint[3]))
    if px >= 0 and px < grid.shape[0] and py >= 0 and py < grid.shape[1] and pz >= 0 and pz < grid.shape[2]:
        for j in range(cell_size):
            if grid[px, py, pz * 2 * cell_size + offset + j] < 0:
                grid[px, py, pz * 2 * cell_size + offset + j] = i
                break
            

TPB = 32

def simulate_rigid_bodies(v1, f1, p1, m1, v2, f2, p2, m2, gridLen, minPt, maxPt):
    
    start = time.time()
    obj1 = PhysObject(v1, f1, gridLen, m1)
    obj2 = PhysObject(v2, f2, gridLen, m2)
    end = time.time()
    print('intiailized objects in',end-start)
    
    print('points in obj1:',obj1.points.shape[0])
    print('points in obj2:',obj2.points.shape[0])
    
    #minPt = np.minimum(np.min(v1, 0), np.min(v2, 0))
    #maxPt = np.maximum(np.max(v1, 0), np.max(v2, 0))
    
    res = np.ceil(((maxPt - minPt) / gridLen)).astype(np.int32)
    
    res[2] *= cell_size * 2
    
    start = time.time()
    
    grid = cuda.to_device(np.full(res, -1, dtype=np.int32))
    #quaternion as [wxyz], followed by translation [xyz]
    pose1 = cuda.to_device(p1)
    pose2 = cuda.to_device(p2)
    points1 = cuda.to_device(obj1.points)
    points2 = cuda.to_device(obj2.points)
    
    end = time.time()
    print('copied data to GPU in',end-start)
    m = obj1.points.shape[0] + obj2.points.shape[0]
    gridDim = (m + TPB - 1) // TPB
    
    starte = cuda.event()
    ende = cuda.event()
    starte.record()
    populate_grid_kernel[gridDim, TPB](points1, pose1, points2, pose2, grid, cuda.to_device(minPt), gridLen)
    ende.record()
    ende.synchronize()
    print('populated grid in',cuda.event_elapsed_time(starte, ende)/1000)
    #debug
    return grid.copy_to_host()
    

In [18]:
pose1 = np.array([1, 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0, 0, 2, 0], np.float32)
grid = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]))

intiailized objects in 0.9714007377624512
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.03889608383178711
populated grid in 0.617333251953125


In [10]:
plot(np.indices(grid.shape)[:,grid >= 0].T)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(54.5, 105…